# Linear Solver Performance: 2D Stokes manufactured Solution after Botti and Di Pietro
### Part 1, Benchmark Setup and Execution

This benchmark was proposed in a work:

 "p‑Multilevel Preconditioners for HHO Discretizations of the Stokes Equations with Static Condensation" 
 by L. Botti and D. Di Pietro (https://doi.org/10.1007/s42967-021-00142-5)
 
It is used to assess DG as well as HDG methods, the latter beeing the main focus of the paper, the former only for reference.
The exact solution to the stationary Stokes equation is
$$
u_1 =  -\exp(x)*(y*\cos(y) + \sin(y)), \\
u_2 =  \exp(x)*y*\sin(y), \\
  p =  2*\exp(x)*\sin(y) .
$$
The domain is set as $ \Omega = (-1,1) $.
This exact solution is also used as a boundary condition on three sides; On one side, a Neumann boundary condition is enforced. It is not mentioned which side this is. 


### Note

This example can be found in the source code repository as as `LinslvPerf_BottiPietroStokes2D.ipynb`. 
One can directly load this into Jupyter to interactively work with the following code examples.

Note: First, BoSSS has to be loaded into the Jupyter kernel. Note:
In the following line, the reference to `BoSSSpad.dll` is required. 
One must either set `#r "BoSSSpad.dll"` to something which is appropirate for the current computer
(e.g. `C:\Program Files (x86)\FDY\BoSSS\bin\Release\net5.0\BoSSSpad.dll` if working with the binary distribution), 
or, if one is working with the source code, one must compile `BoSSSpad`
and put it side-by-side to this worksheet file 
(from the original location in the repository, one can use the scripts `getbossspad.sh`, resp. `getbossspad.bat`).


In [2]:
Console.WriteLine("Execution Date/time is " + DateTime.Now);

Execution Date/time is 09.02.2022 06:49:20


In [3]:
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.XNSFE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


(1,1): error CS0006: Metadata file 'BoSSSpad.dll' could not be found

(5,7): error CS0246: The type or namespace name 'ilPSP' could not be found (are you missing a using directive or an assembly reference?)

(6,7): error CS0246: The type or namespace name 'ilPSP' could not be found (are you missing a using directive or an assembly reference?)

(7,7): error CS0246: The type or namespace name 'BoSSS' could not be found (are you missing a using directive or an assembly reference?)

(8,7): error CS0246: The type or namespace name 'BoSSS' could not be found (are you missing a using directive or an assembly reference?)

(9,7): error CS0246: The type or namespace name 'BoSSS' could not be found (are you missing a using directive or an assembly reference?)

(10,7): error CS0246: The type or namespace name 'BoSSS' could not be found (are you missing a using directive or an assembly reference?)

(11,7): error CS0246: The type or namespace name 'BoSSS' could not be found (are you missing a using

Cell not executed: compilation error

In [ ]:
wmg.Init("LinslvPerf_BottiPietroStokes2D");
wmg.AllJobs

In [ ]:
// for examination on the local workstation, 
//OpenDatabase(@"\\fdygitrunner\ValidationTests\LinslvPerf_XdgStokes");

In [ ]:
/*
// extract the control object to reproduce specific simulation on local workstation
string dest = @"C:\Users\flori\Documents\BoSSS-kummer\public\src\L4-application\XNSE_Solver\bin\Release\net5.0\BenchControls";
foreach(var s in wmg.Sessions) {
    string name = s.Name;
    Console.Write(name + ": ");
    try {
        var ctrl = s.GetControl();
        var txt = ctrl.Serialize();
        
        string DestPath = System.IO.Path.Combine(dest, name + ".obj");
        System.IO.File.WriteAllText(DestPath, txt);
        
        Console.WriteLine("written");
    } catch(Exception e) {
        Console.WriteLine(e.Message);
    }
}
*/

## Utility definitions

In [4]:
static class Utils {
    // DOF per cell for one variable
    public static int Np(int p) {
        return (p*p + 3*p + 2)/2; // 2D 
        //return (p*p*p + 6*p*p + 11*p + 6)/6; // 3D
    }    
    
    /*
    //Non-equidistant nodes
    public static double[] SinLinSpacing(double l, double r, double a, int n) {
        double[] linnodes = GenericBlas.Linspace(-Math.PI * 0.5, Math.PI * 0.5, n);
        double[] linnodes2 = GenericBlas.Linspace(-1, 1, n);
        double[] nodes = new double[n];

        for (int i = 0; i < n; i++)
            //nodes[i] = linnodes2[i] * (1 - a) + (1.0 - Math.Sin(linnodes[i])) * a;
            nodes[i] = linnodes2[i] * (1 - a) + Math.Sin(linnodes[i])*a;

        for (int i = 0; i < n; i++)
            nodes[i] = nodes[i] * (r - l)*0.5 + l;
        return nodes;
    }
    */
}

## Init grids and save to database

In [ ]:
int[] Resolutions_2D = new int[] { 8, 16, 24, 32, 48, 64, 128, 192, 256, 384, 512 };

IGridInfo[] grids = new IGridInfo[Resolutions_2D.Length];
for(int cnt = 0; cnt < Resolutions_3D.Length; cnt++) {
    int Res = Resolutions_3D[cnt];    
    
    double[] xNodes = GenericBlas.Linspace(-1, +1, Res + 1);
    double[] yNodes = GenericBlas.Linspace(-1, +1, Res + 1);
    int J = (xNodes.Length - 1)*(yNodes.Length - 1)*(zNodes.Length - 1);
    
    string GridName = string.Format(wmg.CurrentProject + "_J" + J);
    
    grids[cnt] = wmg.Grids.SingleOrDefault(grd => grd.Name.Contains(string.Format("_J"+ J))); // check if an appropriate grid is already present in the database
    if(grids[cnt] == null){
        Console.WriteLine("Creating grid with " + J + " cells.");
        
        GridCommons g;
        g      = Grid3D.Cartesian3DGrid(xNodes, yNodes, zNodes);
        g.Name = GridName;
        
        g.DefineEdgeTags(delegate (double[] X) {
            return "wall"; // for the moment, ignore the Neumann boundary.
        });
      
        g = wmg.SaveGrid(g);  
        grids[cnt] = g;
    } else {
        Console.WriteLine("Found Grid: " + grids[cnt]);
        if(grids[cnt].NumberOfCells != J)
            throw new Exception("J mismatch");
        
        if(grids[cnt].SpatialDimension != 3)
            throw new Exception("D mismatch");
    }
}

In [ ]:
grids

## Setup Control Object for a Solver Run

In [ ]:
// - - - - - - - - - - - - - - - - - - -
// Initial Values & Boundary conditions
// - - - - - - - - - - - - - - - - - - -

In [ ]:
using BoSSS.Application.XNSE_Solver;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.Timestepping;
using BoSSS.Solution.XdgTimestepping ;

### Setup of Parameter Study

Polynomial degrees to test:

In [ ]:
int[] PolyDegS = new int[] {2, 3, 5};

Solvers which we want to instrument:

In [ ]:
// Solvers which we want to instrument:
LinearSolverCode[] solver_nameS = new LinearSolverCode[] {
    LinearSolverCode.classic_pardiso,
    LinearSolverCode.exp_gmres_levelpmg,
    LinearSolverCode.exp_Kcycle_schwarz,
    LinearSolverCode.exp_Kcycle_ILU
}; 

Maximum Dof for one calculation (we skip fine grids for higher polynomial orders):

In [ ]:
int GetMaxAllowedDOF(LinearSolverCode code) {
    switch(code) {
        case LinearSolverCode.classic_pardiso:
        case LinearSolverCode.classic_mumps:
        return 1100000; // 1.1 Million for iterative solvers at maximum
    
        default: 
        return 5000000; // Up to 5 Million for iterative solvers
    }
}

Loop over all combinations of parameters and define a control object for each combo:

In [ ]:
using BoSSS.Solution.XNSECommon;
using BoSSS.Foundation.XDG;

In [ ]:
List<XNSE_Control> controls = new List<XNSE_Control>();
controls.Clear();
foreach(LinearSolverCode solver in solver_nameS) {
foreach(int k in PolyDegS) {
foreach(IGridInfo grd in grids) {

    int Np_V = Utils.Np(k);
    int Np_p = Utils.Np(k-1);
    int J    = grd.NumberOfCells;
    int DOF  = Np_V*2 + Np_p;
    if(J*DOF >  GetMaxAllowedDOF(solver))
        continue;
    
    // Control Instance, grid, DG degree, etc.
    // =======================================
    
    XNSE_Control C = new XNSE_Control();
    controls.Add(C);
       
    string caseName = string.Format("BottiPietroStokes2D-J{0}_p{1}_{2}", J, k, solver);
    Console.WriteLine("setting up: " + caseName);
    C.SessionName        = caseName;
    
    C.SetGrid(grd);
    C.savetodb = true;
    C.SetDGdegree(k);
    
    // Phys. Parameters
    // ================
    
    // Species A: Water; Species B: Air
    C.PhysicalParameters.rho_A             = 1; // not relevant, since density is not present in steady-state Stokes.
    C.PhysicalParameters.rho_B             = 1; // not relevant, since density is not present in steady-state Stokes.
    C.PhysicalParameters.mu_A              = 1; // dimensionless
    C.PhysicalParameters.mu_B              = 1; // dimensionless
    C.PhysicalParameters.Sigma             = 0; // not relevant, since single phase
    C.PhysicalParameters.IncludeConvection = false;
    C.PhysicalParameters.Material          = true;
    
   
    // Solver Stuff
    // ============
    
    C.VelocityBlockPrecondMode         = MultigridOperator.Mode.SymPart_DiagBlockEquilib;
    
    C.LinearSolver.SolverCode           = solver;
    C.LinearSolver.ConvergenceCriterion = 1e-8;
    C.LevelSet_ConvergenceCriterion     = 1e-6;

    // Timestepping / Instationary
    // ===========================

    C.TimesteppingMode             = AppControl._TimesteppingMode.Steady;
}
}
}

Total number of simulations:

In [ ]:
controls.Count

In [ ]:
// Basic checks on the multigrid configuration
foreach(var ctrl in controls) {
    NUnit.Framework.Assert.Greater(ctrl.NoOfMultigridLevels, 1, "More than 1 multigrid level must be set");
    NUnit.Framework.Assert.Greater(ctrl.LinearSolver.NoOfMultigridLevels, 1, "More than 1 multigrid level must be set");
}

## Launch Jobs

In [ ]:
foreach(var ctrl in controls) {
    Console.WriteLine(" Submitting: " + ctrl.SessionName); 
    ctrl.RunBatch(myBatch);
}

### Wait for Completion and Check Job Status

In [ ]:
wmg.BlockUntilAllJobsTerminate(3600*24*2); // wait at maximum two days for the jobs to finish

In [ ]:
wmg.AllJobs

In [ ]:
wmg.Sessions

In [ ]:
//wmg.Sessions[3].Export().WithSupersampling(2).Do()

In [ ]:
var NoSuccess = wmg.AllJobs.Values.Where(job => job.Status != JobStatus.FinishedSuccessful);
NoSuccess

In [ ]:
// In the case of some failed job, print the directory name for further inspection:
foreach(var fail in NoSuccess)
    Console.WriteLine(fail + ":  @" + fail.LatestDeployment.DeploymentDirectory.FullName);
    //Console.WriteLine(fail.LatestDeployment);

In [ ]:
var FailedSessions = wmg.Sessions.Where(Si => (Si.SuccessfulTermination == false));
FailedSessions

In [ ]:
//foreach(var si in FailedSessions)
//   si.Delete(true);

In [ ]:
NUnit.Framework.Assert.Zero(NoSuccess.Count(), "Some Jobs Failed");

In [ ]:
NUnit.Framework.Assert.Zero(FailedSessions.Count(), "Some Sessions did not terminate successfully.");